In [1]:
import csv
import codecs
import urllib.request
import urllib.error
import sys
import pandas as pd
from datetime import datetime

### Preparar las variables para la URL de la request

In [3]:
games_df = pd.read_csv("csv_files/engineered/new_games_df.csv")


# Sample function to convert and merge date columns
def merge_date_columns(df, date_col, year_col):
    # Define a helper function to convert month name to month number
    def convert_to_date(row):
        # Combine the "MMMM DD" and "YYYY" into one string
        date_str = f"{row[date_col]} {row[year_col]}"
        # Parse the combined string and convert it to "YYYY-MM-DD"
        return datetime.strptime(date_str, "%B %d %Y").strftime("%Y-%m-%d")

    # Apply the function to the dataframe and return a new column with the result
    df["date"] = df.apply(convert_to_date, axis=1)

    return df

games_df = merge_date_columns(games_df, "date", "year")

team_to_city = {
    "Patriots": "Foxborough,MA",
    "Packers": "Green_Bay,WI",
    "Chiefs": "Kansas_City,MO",
    "Cowboys": "Arlington,TX",
    "Rams": "Inglewood,CA",
    "Buccaneers": "Tampa,FL",
    "49ers": "Santa_Clara,CA",
    "Bears": "Chicago,IL",
    "Dolphins": "Miami_Gardens,FL",
    "Jets": "East_Rutherford,NJ",
    "Giants": "East_Rutherford,NJ",
    "Raiders": "Paradise,NV",
    "Broncos": "Denver,CO",
    "Seahawks": "Seattle,WA",
    "Cardinals": "Glendale,AZ",
    "Eagles": "Philadelphia,PA",
    "Bengals": "Cincinnati,OH",
    "Browns": "Cleveland,OH",
    "Steelers": "Pittsburgh,PA",
    "Ravens": "Baltimore,MD",
    "Falcons": "Atlanta,GA",
    "Saints": "New_Orleans,LA",
    "Panthers": "Charlotte,NC",
    "Vikings": "Minneapolis,MN",
    "Lions": "Detroit,MI",
    "Colts": "Indianapolis,IN",
    "Texans": "Houston,TX",
    "Titans": "Nashville,TN",
    "Jaguars": "Jacksonville,FL",
    "Bills": "Orchard_Park,NY",
    "Commanders": "Landover,MD",
    "Redskins": "Landover,MD",
    "Chargers": "Inglewood,CA",
}

games_df["home_city"] = games_df["home_team_name"].map(lambda team: team_to_city[team])

games_df["time"] = pd.to_datetime(games_df["time"], format="%I:%M%p").dt.strftime(
    "%H:%M:%S"
)

print(games_df)

     Unnamed: 0  year  week               winner  day        date      time  \
0             0  2017     1   Kansas City Chiefs  Thu  2017-09-07  20:30:00   
1             1  2017     1      Oakland Raiders  Sun  2017-09-10  13:00:00   
2             2  2017     1      Atlanta Falcons  Sun  2017-09-10  13:00:00   
3             3  2017     1        Buffalo Bills  Sun  2017-09-10  13:00:00   
4             4  2017     1  Pittsburgh Steelers  Sun  2017-09-10  13:00:00   
..          ...   ...   ...                  ...  ...         ...       ...   
763         763  2019    17  Philadelphia Eagles  Sun  2019-12-29  16:25:00   
764         764  2019    17       Dallas Cowboys  Sun  2019-12-29  16:25:00   
765         765  2019    17     Baltimore Ravens  Sun  2019-12-29  16:25:00   
766         766  2019    17     Los Angeles Rams  Sun  2019-12-29  16:25:00   
767         767  2019    17  San Francisco 49ers  Sun  2019-12-29  20:20:00   

     pts_win  pts_loss home_team_name  ...  \
0    

### Generar URL y hacer request para cada fila en el df

In [4]:
BaseURL = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/"

# OBTENER API KEY HACIENDO UNA CUENTA GRATIS EN VISUAL CROSSING
ApiKey = ""
UnitGroup = "metric"

# JSON o CSV
ContentType = "csv"

Include = "current"

def get_weather(row):
    Location = row["home_city"]

    StartDate = row["date"]
    Time = row["time"]

    ApiQuery = BaseURL + Location
    ApiQuery += "/" + StartDate
    ApiQuery += "T"
    ApiQuery += Time
    ApiQuery += "?"
    ApiQuery += "&unitGroup=" + UnitGroup
    ApiQuery += "&contentType=" + ContentType
    ApiQuery += "&include=" + Include
    ApiQuery += "&key=" + ApiKey

    print(" - Running query URL: ", ApiQuery)
    print()

    try:
        CSVBytes = urllib.request.urlopen(ApiQuery)
    except urllib.error.HTTPError as e:
        ErrorInfo = e.read().decode()
        print("Error code: ", e.code, ErrorInfo)
        sys.exit()
    except urllib.error.URLError as e:
        ErrorInfo = e.read().decode()
        print("Error code: ", e.code, ErrorInfo)
        sys.exit()

    # Traducir los resultados a csv
    CSVText = csv.reader(codecs.iterdecode(CSVBytes, "utf-8"))
    CSV_as_list = list(CSVText)
    for row in CSVText:
        print(row)

    city = CSV_as_list[1][0]
    date_time = CSV_as_list[1][1]
    condition = CSV_as_list[1][21]
    return condition

### Ordenar y limpiar informacion devuelta por la API

In [5]:
games_df["weather_condition"] = games_df.apply(get_weather, axis=1)

weather_conditions = {
    "Partially cloudy": "Cloudy",
    "Overcast": "Cloudy",
    "Clear": "Clear",
    "Rain, Partially cloudy": "Rain",
    "Rain, Overcast": "Rain",
    "Rain": "Rain",
    "Snow, Partially cloudy": "Snow",
    "Snow, Overcast": "Snow",
}

games_df["weather_condition"] = games_df["weather_condition"].map(
    lambda condition: weather_conditions[condition]
)

games_df.to_csv("nfl_games_attendance.csv", index=False)

 - Running query URL:  https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Foxborough,MA/2017-09-07T20:30:00?&unitGroup=metric&contentType=csv&include=current&key=P29WFX8F8LHHZ3WHBMFGPXY92

 - Running query URL:  https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Nashville,TN/2017-09-10T13:00:00?&unitGroup=metric&contentType=csv&include=current&key=P29WFX8F8LHHZ3WHBMFGPXY92

 - Running query URL:  https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Chicago,IL/2017-09-10T13:00:00?&unitGroup=metric&contentType=csv&include=current&key=P29WFX8F8LHHZ3WHBMFGPXY92

 - Running query URL:  https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Orchard_Park,NY/2017-09-10T13:00:00?&unitGroup=metric&contentType=csv&include=current&key=P29WFX8F8LHHZ3WHBMFGPXY92

 - Running query URL:  https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Clevela